# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests
import json

url = 'https://en.wikipedia.org/wiki/Data_science'

C:\Users\Sativa\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
html=requests.get(url).content
html[0:600]

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Data science - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XizXXwpAICgAAGoZlnQAAAFR","wgCSPNonce":!1,"wgC'

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [3]:
from bs4 import BeautifulSoup as bs

In [4]:
soup=bs(html,'lxml')
#soup

In [5]:
linkstotales=[e.get('href') for e in soup.find_all('a')]
#linkstotales 

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [6]:
domain = 'http://wikipedia.org'
html=requests.get(domain).content
soup=bs(html,'lxml')
#soup

In [7]:
rel_links=[e.get('href') for e in soup.find_all('a') if (e.get('href')).startswith('//')]
rel_links[0:10]

['//en.wikipedia.org/',
 '//ja.wikipedia.org/',
 '//de.wikipedia.org/',
 '//es.wikipedia.org/',
 '//ru.wikipedia.org/',
 '//fr.wikipedia.org/',
 '//it.wikipedia.org/',
 '//zh.wikipedia.org/',
 '//pt.wikipedia.org/',
 '//pl.wikipedia.org/']

In [8]:
abs_links=[e['href'] for e in soup.select('a[href^="http"]')]
abs_links

['https://en.wikipedia.org/wiki/List_of_Wikipedia_mobile_applications',
 'https://creativecommons.org/licenses/by-sa/3.0/']

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [9]:
import os

In [21]:
folder= 'wikipedia'
#os.mkdir(folder) #crear carpeta
#os.chdir('C:\\Users\\Sativa\\test\\ironhack\\datamex0120\\module-1\\lab-parallelization\\your-code\\wikipedia') #cambiando de directorio de trabajo
#os.getcwd() #comprobando ruta

'C:\\Users\\Sativa\\test\\ironhack\\datamex0120\\module-1\\lab-parallelization\\your-code\\wikipedia'

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [ ]:
#!pip3 install python-slugify 

In [22]:
from slugify import slugify

In [23]:
#con try - except algo andaba mal que no creaba las carpetas
def index_page(link):
    try:
        html=requests.get(link).content
        soup=bs(html,'lxml')
        #print(html)
        
        los_links=[e.get('href') for e in soup.find_all('a')]
        #print(los_links)
        
        lista_slugified=[slugify(e) for e in los_links]
        return lista_slugified
        
    except:
        pass


In [24]:
la_salida=index_page('http://wikipedia.org')
la_salida[0:10]

['en-wikipedia-org',
 'ja-wikipedia-org',
 'de-wikipedia-org',
 'es-wikipedia-org',
 'ru-wikipedia-org',
 'fr-wikipedia-org',
 'it-wikipedia-org',
 'zh-wikipedia-org',
 'pt-wikipedia-org',
 'pl-wikipedia-org']

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [25]:
%%time

for link in la_salida:
    try:
        os.mkdir(link)
    except:
        continue


Wall time: 248 ms


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [26]:
import multiprocessing

In [ ]:
%%time
with multiprocessing.Pool(3) as algo:
    algo.map(index_page,la_salida)